In [1]:
import numpy as np
import scipy as sp
from numpy import array, dot
from numpy.linalg import inv, norm

## Общие сведения

Итерационные методы полезно применять в случае, если не выполняются условия эффективного применения прямых, например, если размерность задачи достаточно велика. Общая идея заключается в том, что вместо поиска самого решения системы $A\mathbf{x}=\mathbf{b}$ строится последовательность векторов $\mathbf{x}^{(0)}, \mathbf{x}^{(1)}, ..., \mathbf{x}^{(k)}...$, о которой заранее известно, что она сходится к истинному решению. Когда элементы последовательности начинают находится достаточно близко друг к другу, итерационный процесс прекращают и в качестве решения системы принимают один из последних элементов построенной последовательности.

Мы будем рассматривать _однослойные_ итерационные методы, т.е. такие, что значение нового элемента последовательности вычисляется с использованием только одного последнего элемента построенной последовательности ($\mathbf{x}^{(k+1)}=F(\mathbf{x}^{(k)})$). Однослойные итерационные методы можно записывать в двух формах:

1) Каноническая: $$B_{k+1}\frac{\mathbf{x}^{(k+1)}-\mathbf{x}^{(k)}}{\tau_{k+1}}+A\mathbf{x}^{(k)}=\mathbf{b}$$

2) Метод последовательных приближений



Здесь можем обозначить $S=E-\tau A$, $\mathbf{f}=\tau \mathbf{b}$



Условия сходимости в общем случае: $\rho(I-\tau A)<1$, удовлетворяется при 
$$ 0 < \tau < \min_i{\frac{2Re\lambda_i(A)}{|\lambda_i(A)|^2}}$$

In [2]:
def run_method(A, b, next_iter, x0, eps, history=False, return_num_iter=False):
    """
    Вспомогательная функция, которая прогоняет метод решения СЛАУ до сходимости.
    При указании параметров, печатает результаты на каждой промежуточной итерации
    или возвращает кол-во итераций.
    """
    x_prev = x0
    x_curr = next_iter(x_prev)
    if history:
        print("k=1. Current approximation: ", x_curr, end='. ')
        print("Error = {}.".format(norm(x_curr - x_prev)))
    k = 1
    while norm(x_curr - x_prev) >= eps:   
        k = k + 1
        x_prev = x_curr
        x_curr = next_iter(x_curr)  
        if history:
            print("k={}. Current approximation: ".format(k), x_curr, end='. ')
            print("Error = {}.".format(norm(x_curr - x_prev)))
    else:
        if history: print('')
    
    if return_num_iter: return x_curr, k
    return x_curr

### Метод простой итерации

Самый простой из всех методов, является прямым (итерационная матрица - единичная) и стационарным.

В классической форме имеет следующий вид:
$$\frac{\mathbf{x}^{(k+1)}-\mathbf{x}^{(k)}}{\tau}+ A \mathbf{x}^{(k)}=\mathbf{b}.$$


В форме метода последовательных приближений:
$$\mathbf{x}^{(k+1)}=(E-\tau A)\mathbf{x}^{(k)}+\tau \mathbf{b}.$$

In [3]:
def simple_iteration_solution(A, b, eps, history=False, return_num_iter=False):
    
    n = A.shape[0]  

    def next_iter(xk):
        from numpy import identity as E
        nonlocal n, A, b
        return dot((E(n)-tau0*A), xk) + tau0*b
    
    return run_method(A, b, next_iter, tau0*b, eps, history, return_num_iter)

### Метод Якоби

In [4]:
def jacobi_solution(A, b, eps, history=False, return_num_iter=False):
    AL = np.tril(A, k=-1)
    D = np.where(np.identity(len(A))==1, A, 0)
    AU = np.triu(A, k=1)
    D_inv = np.diag(1 / np.diag(D))

    def next_iter(xk):
        nonlocal AL, AU, D_inv, A, b
        return -np.dot(np.dot(D_inv, AL+AU), xk) + np.dot(D_inv, b)
        
    return run_method(A, b, next_iter, tau0*b, eps, history, return_num_iter)

### Метод Зейделя

In [5]:
def seidel_solution(A, b, eps, history=False, return_num_iter=False):
    # initialize tau0
    AL_plus_D = np.tril(A)
    AU = np.triu(A, k=1)
    AL_plus_D_inv = inv(AL_plus_D)
    
    def  next_iter(xk):
        nonlocal AL_plus_D_inv, AU, b
        return -np.dot(np.dot(AL_plus_D_inv, AU), xk) + np.dot(AL_plus_D_inv, b)
        
    return run_method(A, b, next_iter, tau0*b, eps, history, return_num_iter)

### Метод минимальных невязок 

In [6]:
def min_residuals_solution(A, b, eps, history=False, return_num_iter=False):
    
    def next_iter(xk):
        nonlocal A, b
        rk = dot(A, xk) - b
        Ark = dot(A, rk)
        return xk - dot(Ark, rk) / dot(Ark, Ark) * rk        
        
    return run_method(A, b, next_iter, tau0*b, eps, history, return_num_iter)   

### Метод минимальных поправок 

In [7]:
def min_correction_solution(A, b, eps, history=False, return_num_iter=False):
    
    B_inv = np.diag(1 / np.diag(A))
    
    def next_iter(xk):
        nonlocal A, b, B_inv
        rk = dot(A, xk) - b
        wk = np.dot(B_inv, rk)
        Awk = np.dot(A, wk)
        vk = np.dot(B_inv, Awk)
        return xk - np.dot(Awk, wk) / np.dot(vk, Awk) * wk
        
    return run_method(A, b, next_iter, tau0*b, eps, history, return_num_iter)  

### Метод наискорейшего спуска 

In [8]:
def fastest_descent_solution(A, b, eps, history=False, return_num_iter=False):
    
    def next_iter(xk):
        nonlocal A, b
        rk = dot(A, xk) - b
        Ark = dot(A, rk)
        return xk - np.dot(rk, rk) / np.dot(Ark, rk) * rk
    
    return run_method(A, b, next_iter, tau0*b, eps, history, return_num_iter) 

### Пример и сравнение скорости сходимости методов 

In [9]:
A = array([
    [ 3.77, 0.56, -1.45, 1.66],
    [ 0.56, 9.56,  5.23, 1.62],
    [-1.45, 5.23, 12.87, 1.99],
    [ 1.66, 1.62,  1.99, 6.89] 
])
 
b = array([8.94, -7.15, 10.45, 9.11])

x_true = dot(inv(A), b)
print("The exact solution: ", x_true)

The exact solution:  [ 3.22167612 -2.08251357  1.94807228  0.4730081 ]


In [14]:
import numpy as np
phi = np.arctan(2*A[0, 1] / (A[0, 0] - A[1, 1])) / 2

C:\Users\lekho\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


In [8]:
U0 = np.array([
    [np.cos(phi), -np.sin(phi)],
    [np.sin(phi),  np.cos(phi)]
])
U0

array([[ 0.96371493,  0.26693358],
       [-0.26693358,  0.96371493]])

In [15]:
A = np.array([
    [4, -0.3],
    [-0.3, 3]
])

np.dot(U0.T, np.dot(A, U0))

array([[  4.08309519e+00,   0.00000000e+00],
       [ -2.22044605e-16,   2.91690481e+00]])

In [13]:
np.linalg.eig(A)

(array([ 4.3,  3.7]), array([[ 0.70710678,  0.70710678],
        [-0.70710678,  0.70710678]]))

In [10]:
# число обусловленности матрицы
# по умолчанию - отношение максимального
# и минимального собственных значений
print(np.linalg.cond(A))

# если считать самим по норме
# print(dot(A.T, A))
ev = np.linalg.eigvals(dot(A.T, A))
print(ev)
R = np.sqrt(ev.max())
#print(R)
ev_inv = np.linalg.eigvals(dot(inv(A).T, inv(A)))
print(ev_inv)
r = np.sqrt(ev_inv.max())
#print(r)
print(R * r)

# тоже самим, но как отношение максимального и минимального с.ч.
evA = np.linalg.eigvals(A)
print(evA)
lmax = evA.max()
lmin = evA.min()
print(lmax / lmin)
xi = lmin / lmax
rho = (1 - xi) / (1 + xi)
xi, rho

7.14258701334
[ 300.77669139    5.89566907   56.90744652   33.35389302]
[ 0.16961603  0.02998151  0.01757239  0.00332473]
7.14258701334
[ 17.34291473   2.42809989   7.54370244   5.77528294]
7.14258701334


(0.14000529473883958, 0.75437781669090764)

In [11]:
np.log(10**4) / np.log(1 / rho)

32.676777632324061

In [12]:
tau_max = 2 / lmax
tau0 = 2 / (lmax + lmin)

In [13]:
tau0 * b

array([ 0.90435419, -0.72328104,  1.05710306,  0.92155109])

In [14]:
def print_method_results(A, b, eps, meth):
    x_true = dot(inv(A), b)
    x_appr, k = meth(A, b, 1e-4, history=False, return_num_iter=True)
    print("{} method, # of iterations = {}.".format(meth.__name__, k))
    print("Aproximate solution: {}.".format(x_appr))
    print("Error = {}.".format(norm(x_appr - x_true)))
    print("Residual = {}.\n".format(norm(dot(A, x_appr)-b)))

In [15]:
for meth in [simple_iteration_solution, jacobi_solution, 
             seidel_solution, min_residuals_solution, 
             min_correction_solution, fastest_descent_solution]:
    
    print_method_results(A, b, 1e-4, meth)

simple_iteration_solution method, # of iterations = 33.
Aproximate solution: [ 3.22146697 -2.08248003  1.94798345  0.47309496].
Error = 0.00024556693354028014.
Residual = 0.0007208550017071671.

jacobi_solution method, # of iterations = 20.
Aproximate solution: [ 3.22161433 -2.08246354  1.94803144  0.47304974].
Error = 9.861052004139195e-05.
Residual = 0.0003697627714193037.

seidel_solution method, # of iterations = 11.
Aproximate solution: [ 3.22163792 -2.08248917  1.9480545   0.4730167 ].
Error = 4.944317771032854e-05.
Residual = 0.00016315883345091386.

min_residuals_solution method, # of iterations = 33.
Aproximate solution: [ 3.2214591  -2.08247186  1.94798979  0.4731009 ].
Error = 0.000253482970391598.
Residual = 0.0006506547249753918.

min_correction_solution method, # of iterations = 17.
Aproximate solution: [ 3.22156126 -2.08245048  1.94798348  0.47305894].
Error = 0.00016626367303081824.
Residual = 0.0006025590479273847.

fastest_descent_solution method, # of iterations = 28